In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.cm as cm

import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('.../bats-code')
from bat_functions import get_bat_accumulation

In [ ]:
day_folders

In [ ]:
day_folders = sorted(glob.glob('.../kasanka-bats/processed/deep-learning/*Nov'))
observations = {}
for day_folder in day_folders[:2]:
    print(day_folder)

    date = os.path.basename(day_folder)
    track_files = sorted(glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy')))
    observations[date] = {}
    for track_file in track_files: 
        camera = track_file.split('/')[-2]
        obs = {'date': date,
               'camera': camera,
               }
        crossing_tracks_list = np.load(track_file, allow_pickle=True)
        passing_bat_frame = []
        passing_bat_size = []
        passing_bat_track_ind = []
        passing_bat_direction = []
        passing_bat_track_id = []
        for track_ind, track in enumerate(crossing_tracks_list):
            if track['crossed'] > 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(1)

            elif track['crossed'] < 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(-1)

        obs['frames'] = np.array(passing_bat_frame)
        obs['mean_wing'] = np.array(passing_bat_size)
        obs['ids'] = np.array(passing_bat_track_id)
        obs['direction'] = np.array(passing_bat_direction)
        observations[date][camera] = obs

In [ ]:
for date, day_obs in observations.items():
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'])
        observations[date][cam_name]['accumulation'] = bat_accumulation

In [ ]:
percentiles = [.65, .8, .95, .98]

In [ ]:
camera_name = None

tracks_files = []

for date in ['16Nov', '17Nov']:

    tracks_folder = os.path.join('.../kasanka-bats/processed', 'deep-learning', date) 
    frame_root_folder = '.../bat-videos/raw-falloff-validation-videos' 

    if camera_name:
        tracks_files.extend(
            glob.glob(os.path.join(tracks_folder, camera_name, 'raw_tracks.npy'))
        )
    else:
        tracks_files.extend(
            glob.glob(os.path.join(tracks_folder, '*/raw_tracks.npy'))
        )

In [ ]:
def check_track_crossing(track, frame_num, line_height):
    """ Check is a given bat is crossing line in given frame.
    Return -1 if coming back, 1 is leaving
    
    Track: track object
    frame_num: frame to investigate
    line_height: what counts as crossing
    """
    if track['first_frame'] > frame_num:
        # Not in frame
        return 0
    if track['first_frame'] + track['track'].shape[0] <= frame_num:
        # track already gone
        return 0
    
#     print(frame_num, track['first_frame'], track['track'].shape)
    
    rel_frame_num = frame_num - track['first_frame']
    next_frame = np.min([rel_frame_num+1, track['track'].shape[0]-1])

    if track['track'][rel_frame_num, 1] <= line_height:
        # Could be coming back
        if track['track'][next_frame, 1] > line_height:
            # Is coming back
            return -1 
    if track['track'][rel_frame_num, 1] >= line_height:
        # Could be leaving
        if track['track'][next_frame, 1] < line_height:
            # Is leaving
            return 1
    else:
        return 0

def draw_crossing_dots(frame, crossings, y_line_height):
    """Draw all the active crossing dots on given frame"""
    
    cv2.line(frame, (0, int(y_line_height)), (frame.shape[1], int(y_line_height)), (0,0,255), 1)
    
    next_crossings = [] # list of ones that will be drawn still in next frame
    
    for crossing in crossings:
        if crossing['type'] == 'coming':
            color = (0, 255, 255)
        if crossing['type'] == 'going':
            color = (255, 255, 0)
        cv2.circle(frame, (int(crossing['position'][0]), int(crossing['position'][1])), crossing['life']*4, color, 2)
            
        crossing['life'] -= 1
        if crossing['life'] > 0:
            next_crossings.append(crossing)
            
    return frame, next_crossings

def draw_all_dots(frame, frame_ind, tracks):
    """Draw all the active dots on given frame"""
    
    cmap = cm.get_cmap('hsv')
    
    num_colors = 30
    
    for track_ind, track in zip(np.arange(len(tracks))[::-1],tracks[::-1]):
        if track['first_frame'] > frame_ind:
            continue
        if track['first_frame'] + track['track'].shape[0] <= frame_ind:
            continue
            
        c = np.array(cmap((track_ind%num_colors)/num_colors))[:3] * 255
        c = (int(c[0]), int(c[1]), int(c[2]))
            
        track_rel_frame = frame_ind - track['first_frame']
        cv2.circle(frame, (int(track['track'][track_rel_frame, 0]), int(track['track'][track_rel_frame, 1])), 
                   3, c, -1)
#         cv2.circle(frame, (int(track['track'][track_rel_frame, 0]), int(track['track'][track_rel_frame, 1])), 
#                    2, (c[0], c[1], c[2]), -1)

            
    return frame
            
    
def create_clip(frame_files, tracks, first_frame, line_height, output_file, 
                scale=False, draw_padding_box=False):
    
    dot_life = 5 # how many frames to show crossing dots for
    padding_shift = 48
    
    demo_frame = cv2.imread(frame_files[0])
    if raw:
        out = cv2.VideoWriter(output_file, 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          30, 
                          (demo_frame.shape[1]-2*padding_shift, 
                           demo_frame.shape[0]-2*padding_shift
                          )
                         )
    else: 
        out = cv2.VideoWriter(output_file, 
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              30, 
                              (demo_frame.shape[1], demo_frame.shape[0])
                             )
    
    crossings = []
    
    line_height = line_height * demo_frame.shape[0]
    
    for frame_num, file in enumerate(frame_files):
        frame = cv2.imread(file)
        if tracks is None:
            # Just make raw video
            out.write(frame[padding_shift:-padding_shift, padding_shift:-padding_shift])
            continue
            
        if scale:
            frame = frame.astype(float) / np.max(frame)
            frame *= 255
            frame = frame.astype(np.uint8)
        for track in tracks:
            crossing = check_track_crossing(track, first_frame+frame_num, line_height)
            if crossing == -1:
                crossings.append({'position': track['track'][first_frame+frame_num-track['first_frame']],
                                  'life': dot_life,
                                  'type': 'coming'})
            if crossing == 1:
                crossings.append({'position': track['track'][first_frame+frame_num-track['first_frame']],
                                  'life': dot_life,
                                  'type': 'going'})
        frame = draw_all_dots(frame, frame_num+first_frame, tracks)

        frame, crossings = draw_crossing_dots(frame, crossings, line_height)
        
        if draw_padding_box:
            
            frame_height = frame.shape[0]
            frame_width = frame.shape[1]
            cv2.line(frame, 
                     (padding_shift, padding_shift), 
                     (padding_shift, frame_height - padding_shift), 
                     (255, 255, 255), 
                     1)
            cv2.line(frame, 
                     (padding_shift, frame_height - padding_shift), 
                     (frame_width - padding_shift, frame_height - padding_shift), 
                     (255, 255, 255), 
                     1)
            cv2.line(frame, 
                     (frame_width - padding_shift, frame_height - padding_shift), 
                     (frame_width - padding_shift, padding_shift), 
                     (255, 255, 255), 
                     1)
            cv2.line(frame, 
                     (frame_width - padding_shift, padding_shift), 
                     (padding_shift, padding_shift), 
                     (255, 255, 255), 
                     1)
            
            cv2.line(frame, 
                     (int(frame_width / 3), 0), 
                     (int(frame_width / 3), frame_height), 
                     (255, 255, 255), 
                     1)
            cv2.line(frame, 
                     (int(2*frame_width / 3), 0), 
                     (int(2*frame_width / 3), frame_height), 
                     (255, 255, 255), 
                     1)
        
                
        out.write(frame)
    out.release()

In [ ]:
# tracks_files = tracks_files[:2] + tracks_files[3:]
tracks_files

In [ ]:
observation_name = 'FibwePublic'
frame_files = sorted(glob.glob(os.path.join(frame_root_folder, observation_name, '*/*.jpg')))
len(frame_files)

In [ ]:
positions_file = f'.../kasanka-bats/processed/deep-learning/{date}/{observation_name}/centers.npy'
positions = np.load(positions_file, allow_pickle=True)

In [ ]:
len(frame_files)
frame_files = frame_files

In [ ]:
len(frame_files) - len(positions)
# len(positions)
# frame_files = frame_files[len(frame_files) - len(positions):]

In [ ]:
frame_ind = 55000
plt.figure(figsize=(20,20))
plt.imshow(plt.imread(frame_files[frame_ind]))
# rel_ind = frame_ind - (len(frame_files) - len(positions))
plt.scatter(positions[frame_ind][:,0], positions[frame_ind][:,1], s=3, c='r')

In [ ]:
# frame_files[-100:]

In [ ]:
tracks = np.load(tracks_files[5], allow_pickle=True)

In [ ]:
frame_ind = 5000
plt.figure(figsize=(20,20))
plt.imshow(plt.imread(frame_files[frame_ind]))
for t in tracks:
    if t['first_frame'] < frame_ind:
        if t['last_frame'] > frame_ind:
            rel_ind = frame_ind - t['first_frame']
            plt.scatter(t['track'][rel_ind, 0], t['track'][rel_ind, 1], s=1)

In [ ]:
frame = plt.imread(frame_files[0])
frame_width = frame.shape[1]
frame_height = frame.shape[0]
cv2.line(frame, 
                     (int(frame_width / 3), 0), 
                     (int(frame_width / 3), frame_height), 
                     (255, 255, 255), 
                     1)
plt.imshow(frame)

In [ ]:
tracks_files[:1]

In [ ]:
clip_time = 7 # in seconds (30fps)

raw = True

for track_ind, track_file in enumerate(tracks_files[1:]):
#     if 'FibweParking2' in track_file:
#         continue

    if track_ind % 5 == 0:
        print(f'{track_ind} observations proccesed')
    observation_name = track_file.split('/')[-2]
    date = track_file.split('/')[-3]
    frame_files = sorted(glob.glob(os.path.join(frame_root_folder, date, 
                                                observation_name, '*/*.jpg')))
#     frame_files = frame_files[805:]
    line_height = .5
    
    output_folder = os.path.join(root_output_folder, date, observation_name)
    os.makedirs(output_folder, exist_ok=True)
    
    
    accumulation = observations[date][observation_name]['accumulation']
    accumulation_percent = accumulation / accumulation[-1]
    plt.figure()
    plt.plot(accumulation_percent)
    first_frames = []
    for percentile in percentiles:
        obs_ind = np.argmax(accumulation_percent > percentile)
        plt.scatter(obs_ind, accumulation_percent[obs_ind])
        first_frames.append(obs_ind)
    plt.title(f'{observation_name}')
    
#     first_frames = np.linspace(0, len(frame_files) - clip_time*30-1, num=6, endpoint=True, dtype=int)
    if raw:
        tracks = None
    else:
        tracks = np.load(track_file, allow_pickle=True)
    
    for clip_num, first_frame in enumerate(first_frames[:]):
        last_frame = first_frame + int(30 * clip_time)
        video_file = os.path.join(output_folder, 'test-camera-{}-clip-{}-firstframe-{}-cliptime-{}-scaled.mp4'.format(
            observation_name, clip_num, first_frame, clip_time))
        if raw:
            create_clip(frame_files[first_frame:last_frame], tracks, 
                        first_frame, line_height, video_file, scale=True,
                        draw_padding_box=False)
        else:
            create_clip(frame_files[first_frame:last_frame], tracks, 
                        first_frame, line_height, video_file, scale=True,
                        draw_padding_box=True)
        

In [ ]:
import pandas as pd

In [ ]:
video_files = sorted(
    glob.glob(os.path.join(root_output_folder, '*', '*'))
)
print(*video_files[:10], sep='\n')

In [ ]:
filenames = [os.path.basename(f) for f in video_files]
validation_df = pd.DataFrame({'date-folder': date,
                              'video-clip-name': filenames,
                              })
validation_df['number-missed-detections-going'] = None
validation_df['number-missed-detections-coming'] = None
validation_df['number-false-detections-going(blue)'] = None
validation_df['number-false-detections-coming(yellow)'] = None
validation_df['name-of-checker'] = None
validation_df['notes'] = None

In [ ]:
save_folder = '.../bat-videos/new-method-fall-off'
file_name = 'falloff-validation-17Nov.csv'
validation_df.to_csv(os.path.join(save_folder, file_name), index=False)
validation_df

In [ ]:
type(plt.imread(frame_files[0])[0,0,0])

In [ ]:
import matplotlib.cm as cm

cmap = cm.get_cmap('hsv')
c = np.array(cmap(.01))[:3] * 255
c = tuple(c.astype(int))
type(c[0])

In [ ]:
image = cv2.imread(frame_files[20000])

frame = draw_all_dots(image, 20000, tracks)

In [ ]:
cv2.imshow('frame', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
c = (255,0,255)
type(c[0])